# Tensor Network Theory - Lab Class
## Notebook 1: Basic tensor manipulations

We start all our notebooks by including the libraries we need. Here we will use ITensors:

In [2]:
using ITensors

In this notebook we will explore creating, manipulating and contracting tensors in ITensor. A tensor is effectively a multi-dimensional array. In ITensor the indices of this array are themselves objects containing with a unique identifier, a dimension and a tag which gives them a sensible name. 

### Indices 
Let's create four indices:

In [78]:
i = Index(3,"i") # 3 dimensional index called i
j = Index(3,"j") # 3 dimensional index called j
k = Index(4,"k") # 4 dimensional index called k
l = Index(3,"l") # 3 dimensional index called l

(dim=3|id=676|"l")

The output from the last line (the only one shown in a cell) confirms that index $l$ has the properties we wanted. Using the first three of these indices we can now create a variety of tensors, for example:

<img src="./images/tensor_diagrams.png" alt= “” width="600" height="300">

Now let's create each of these tensors as:

In [135]:
v = ITensor(j);
M = ITensor(i,j);
T = ITensor(i,j,k);

We suppressed the output here, but we can confirm for example that $T$ it is an order-3 tensor by using the inds(.) function to examine it:

In [80]:
inds(T)

((dim=3|id=459|"i"), (dim=3|id=155|"j"), (dim=4|id=880|"k"))

### Populating tensors
The tensors we have created are intialised to zero. Let's populate this tensor with some specific values. We do this via an assignment:

In [136]:
v[j=>2] = 1.12;
M[i=>1,j=>1] = 45;
T[i=>2,j=>1,k=>3] = -3.2;

We can check this has worked as:

In [137]:
T[j=>1,i=>2,k=>3]

-3.2

Notice the evaluation above specifies the indices in a different order to the assignment. The illustrates that since we pass indices the order we write them in is irrelevant. We can convert a tensor into a conventional multi-dimensional array in Julia as:

In [83]:
Array(v,j)

3-element Vector{Float64}:
 0.0
 1.12
 0.0

In [138]:
Array(M,i,j)

3×3 Matrix{Int64}:
 45  0  0
  0  0  0
  0  0  0

Here we had to pass the indices and the order matters. We get an array with a different order of dimensions if we switch them around:

In [85]:
Array(M,j,i)

3×3 Matrix{Int64}:
  0  0  0
 45  0  0
  0  0  0

---

## *Exercise*
- Try `Array(T,i,j,k)` and `Array(T,j,i,k)`.

---

We can also create tensor with specific elements by passing arrays as:

In [113]:
Q = ITensor([0 1.0 0; -1.0 0 0; 0 0 1.0],i,j);
Array(Q,i,j) # Check we get the correct matrix

3×3 Matrix{Float64}:
  0.0  1.0  0.0
 -1.0  0.0  0.0
  0.0  0.0  1.0

We can also generate random array and assign this to a tensor:

In [114]:
Yarr = randn(3,4,3)
Y = ITensor(Yarr,j,k,i);

The indices should be listed so they coincide with the dimensions of the array they index over. We can check by examining whether elements are identical between the tensor and array:

In [115]:
isapprox(Y[k=>1,i=>2,j=>3],Yarr[3,1,2])

true

---

## *Exercise*
- Try `Yd = ITensor(Yarr,i,j,k)`. This code works despite the incorrect index assignments to array dimensions. What is it doing?

---


### Arithmetic and elementwise operations on tensors
We can do arithmetic with tensors once they share the same collection of indices (the order they appear in a tensor is again irrelevant). For example we can multiply by complex scalars and add up tensors:

In [119]:
X = 2.0*T + Y/(1+1im);

We can check an element to see if it has been added up:

In [120]:
isapprox(X[j=>1,i=>2,k=>3],2.0*T[j=>1,i=>2,k=>3] + Y[j=>1,i=>2,k=>3]/(1+1im))

true

We can also do elementwise operations on tensors, like multiplication by a scalar and broadcasting functions:

In [91]:
C1 = abs.(Q); # Take the abs() of all elements
C2 = one.(Q); # Replace all elements by 1

You can define more complex functions yourself as:

In [92]:
myfunc(x) = 1.0/(1.0+exp(-x));

and then broadcast these over all elements of a tensor:

In [93]:
C3 = myfunc.(Q);

---

## *Exercise*
- Check the arrays from the tensors `C1`, `C2`, and `C3` to see if they are as expected.

---

### Special tensors

There are shortcuts to creating some special tensors:

In [121]:
A = randomITensor(i,j)   # Create a random order-2 tensor (3x3 matrix)
B = randomITensor(j,k)   # Create another random order-2 tensor (3x4 matrix)
N = randomITensor(j,l,k) # Create a random order-3 tensor (3x3x4 array)
P = onehot(i=>3,j=>2)    # Create a tensor with a single unit element (3x3 matrix)
D = delta(i,j);          # Create a diagonal identity tensor (3x3 matrix)

It is useful to examine the output of the last two:

In [95]:
Array(P,i,j)

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  1.0  0.0

The identity tensor has a special diagram comprised of just a line:
<img src="./images/identity_diagram.png" alt= “” width="300" height="300">
and we can confirm its elements:

In [97]:
Array(D,i,j)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Sometimes we might want to combine two indices into one larger index. We can create a combiner tensor to do this:

In [102]:
C = combiner(i,k; tags="c")

ITensor ord=3 (dim=12|id=517|"c") (dim=3|id=459|"i") (dim=4|id=880|"k")
NDTensors.Combiner

As expected this is an order-3 tensor with a new 12 dimensional index c. We can access this new index object by using the following command:

In [103]:
c = combinedind(C)

(dim=12|id=517|"c")

### Contracting tensors

Contraction of tensor indices means a sum of a multiplications of elements. We represent this diagrammatically as:
<img src="./images/sample_contractions.png" alt= “” width="600" height="300">

which corresponds to matrix x vector and matrix x matrix operations. We can use the $*$ operation and ITensor will automatically contract the shared index, which in both cases is $j$:

In [122]:
M*v

ITensor ord=1 (dim=3|id=155|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [123]:
A*B

ITensor ord=2 (dim=3|id=459|"i") (dim=4|id=880|"k")
NDTensors.Dense{Float64, Vector{Float64}}

Contraction generalises beyond linear algebra. Consider the tensor $M$ is order-2 and the tensor $N$ is order-3, but they share the $i$ index. The contraction of these tensors is expressed diagrammatically as:

<img src="./images/generic_diagram.png" alt= “” width="600" height="300">

We are left with an order-3 tensor with the remaining $i$, $l$ and $k$ indices:

In [124]:
M*N

ITensor ord=3 (dim=3|id=459|"i") (dim=3|id=676|"l") (dim=4|id=880|"k")
NDTensors.Dense{Float64, Vector{Float64}}

If we $*$ tensors with no shared indices we simply get the outer product
<img src="./images/outer_product.png" alt= “” width="350" height="300">

In [111]:
w = randomITensor(j) # Create a random vector with index j
T = v*w              # Overwrite T as the outer product

ITensor ord=2 (dim=3|id=459|"i") (dim=3|id=155|"j")
NDTensors.Dense{Float64, Vector{Float64}}

The action of the identity tensor can also be demonstrated:
<img src="./images/identity_contraction.png" alt= “” width="700" height="300">

In [128]:
T = randomITensor(i,j,k)  # Overwrite T as a random order-3 tensor
T = T*delta(j,l)          # Now index j and is replaced by l

ITensor ord=3 (dim=3|id=459|"i") (dim=4|id=880|"k") (dim=3|id=676|"l")
NDTensors.Dense{Float64, Vector{Float64}}

We haven't changed the tensor $T$ aside from replace the index $j$ with the index $l$. We can use the identity tensor to perform partial tracing of indices:
<img src="./images/trace_A.png" alt= “” width="700" height="300">

In [130]:
T = randomITensor(i,j,l)  # Overwrite T as a random order-3 tensor again
T = T*delta(i,l)

ITensor ord=1 (dim=3|id=155|"j")
NDTensors.Dense{Float64, Vector{Float64}}

Likewise we can do a full matrix trace as:
<img src="./images/trace.png" alt= “” width="300" height="300">
which leaves a rank-0 tensor = scalar:

In [139]:
s = M*delta(i,j)

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [140]:
Array(s)

0-dimensional Array{Float64, 0}:
45.0

If we redefine $T$ as an order-3 tensor again and contract with the combiner $C$ then we get an order-2 tensor with the $i$ and $k$ legs combined: 
<img src="./images/combiner_itensor.png" alt= “” width="700" height="300">

In [126]:
T = randomITensor(i,j,k)  # Overwrite T as a random order-3 tensor again 
F = T*C

ITensor ord=2 (dim=3|id=155|"j") (dim=12|id=517|"c")
NDTensors.Dense{Float64, Vector{Float64}}

We are left we an order-2 tensor with indices $j$ and $c$ which we can examine as a 3 x 12 array:

In [106]:
Array(F,j,c)

3×12 Matrix{ComplexF64}:
 -0.680676+0.680676im   -0.739827+0.739827im  …  -0.627573+0.627573im
 0.0395894-0.0395894im   0.289936-0.289936im     -0.581361+0.581361im
 -0.210137+0.210137im     1.16727-1.16727im      -0.637969+0.637969im

---

## *Exercise*
- Define the following a new random order-4 tensor $T$, a new index $m$, and a random order-2 tensor $V$. Then perform the following contraction:
<img src="./images/tensor_contraction.png" alt= “” width="500" height="300">
---

### Decomposing tensors

We can employ standard linear algebra matrix decompositions to tensors by treating collections of indices as a "row" and "column" indices of a matrix. A very well used example inside tensor network algorithms is the SVD:

<img src="./images/svd.png" alt= “” width="700" height="300">

In [146]:
T = randomITensor(i,j,k)  # Define a new random order-3 tensor T
U,S,V = svd(T,(i,k))      # Perform the SVD treating the indices $i$ and $k$ as the row index
@show norm(U*S*V - T);    # We are guaranteed that U*S*V contraction will yield T to numerical precision

norm(U * S * V - T) = 2.1662313824139103e-15


The QR decomposition works similarly:
<img src="./images/qr.png" alt= “” width="700" height="300">

In [147]:
T = randomITensor(i,j,k)        # Define a new random order-3 tensor T
Q,R = qr(T,(i,k);positive=true) # Perform the QR treating the indices $i$ and $k$ as the row index
@show norm(Q*R - T);            # We are guaranteed that U*R contraction will yield T to numerical precision

norm(Q * R - T) = 6.944811124780224e-16


---
### Acknowledgement:
The examples and tensor diagrams used in this notebook have been taken directly from the [ITensors.jl](https://itensor.github.io/ITensors.jl/stable/examples/ITensor.html) documentation!
<img src="./images/itensor_logo.png" alt= “” width="100" height="300">
---